## Subarray sum equal to target

Given an array of integers and a target:
1. Check if any subarray sums to the target, return True in that case else return False
2. Find the count of all possible subarrays that sum to the target
3. Find the index pairs (i, j) of the first subarray
4. Find the list of index pairs (i, j) of all the possible subarrays

TRICK: The idea is to use *prefix sum*. If the prefix sum till a certain
index *i* is *p* and the prefix sum till another index *j* is *p + target*, then 
we know that the subarray (i, j) sums to the target. Or vice-versa if the prefix 
sum at index *j* is *p* and prefix sum till a certain lower index *i* was *p - target*, 
then the subarray (i, j) sums to the target. To do this check, we will use a hashmap.

In [1]:
from typing import List, Tuple, Dict, Any, Optional
from collections import defaultdict

In [2]:
def subarray_sum_varient_1(nums: List[int], target: int) -> bool:
    last_prefix_sum = 0
    hashmap = set()

    for i, num in enumerate(nums):
        last_prefix_sum += num
        if last_prefix_sum in hashmap:
            return True
        hashmap.add(last_prefix_sum + target)
    return False


def subarray_sum_varient_2(nums: List[int], target: int) -> int:
    last_prefix_sum, count = 0, 0
    hashmap = set()

    for i, num in enumerate(nums):
        last_prefix_sum += num
        if last_prefix_sum in hashmap:
            count += 1
        hashmap.add(last_prefix_sum + target)
    return count


def subarray_sum_varient_3(nums: List[int], target: int) -> List[int]:
    last_prefix_sum = 0
    hashmap = {}

    for i, num in enumerate(nums):
        last_prefix_sum += num
        if last_prefix_sum in hashmap:
            return [hashmap[last_prefix_sum] + 1, i]
        hashmap[last_prefix_sum + target] = i
    return [-1, -1]
    

def subarray_sum_varient_4(nums: List[int], target: int) -> List[List[int]]:
    last_prefix_sum = 0
    hashmap = {}

    all_pairs = []
    for i, num in enumerate(nums):
        last_prefix_sum += num
        if last_prefix_sum in hashmap:
            pairs = [[ele + 1, i] for ele in hashmap[last_prefix_sum]]
            all_pairs.extend(pairs)

        if last_prefix_sum + target in hashmap:
            hashmap[last_prefix_sum + target].append(i)
        else:
            hashmap[last_prefix_sum + target] = [i]
    return all_pairs

In [3]:
heights = [1, 5, -2, 10, 3, 4]
target = 3

print(subarray_sum_varient_1(nums=heights, target=target))
print(subarray_sum_varient_2(nums=heights, target=target))
print(subarray_sum_varient_3(nums=heights, target=target))
print(subarray_sum_varient_4(nums=heights, target=target))

True
2
[1, 2]
[[1, 2], [4, 4]]


## SubMatrix sum

We are given a matrix of dimensions *m x n* and a target and the goal is to return:
1. Number of submatrices that sum to the target

A brute-force approach would be to take all the possible submatrices and check 
if the sum of its elements equals the target. A submatrix can be uniquely 
represented by four elements: the coordinates of the top-left position (x1, y1)
and the coordinates of the bottom-right position (x2, y2). And to sum the elements
we would have to go over the (x2 - x1) * (y2 - y1) elements. Thus in total the time
complexity is going to be $O(M^3 N^3)$.

To optimize the summing of elements in a submatrix, we can make use of the prefix
sum matrix $K$. A prefix sum at position (x, y) is the sum of all elements given by the 
region (0, 0) and (x, y). Thus the sub-matrix sum can be written as:

$$sum [(x1, y1), (x2, y2)]  = K(x2, y2) - K(x1-1, y2) - K(x2, y1-1) + K(x1-1, y1-1)$$

Once we have the prefix sum matrix computed, we can go over all the pairs of submatrices, 
compute the sum of its elements using the above formula in $O(1)$ time and check if it 
matches the target, bringing the time complexity down to $O(M^2 N^2)$. Using a hashmap 
for one of the indices as we did for the subarray sum, we can bring it further down to $O(M^2N)$.


In [4]:
def submatrix_sum(matrix: List[List[int]], target: int) -> int:
    m, n = len(matrix), len(matrix[0])

    prefix_sum = [[0] * (n+1) for _ in range(m+1)]
    for i in range(1, m+1):
        for j in range(1, n+1):
            prefix_sum[i][j] = (
                prefix_sum[i - 1][j] +
                prefix_sum[i][j - 1] -
                prefix_sum[i-1][j-1] +
                matrix[i-1][j-1]
            )
    
    # Now that we have computed all prefix sums, 
    # we go through each submatrix given by top left corner (x1, y1)
    # and bottom right corner (x2, y2) and check if the sum of elements
    # in this matrix sum to the target; using the prefix sum computed.
    count = 0
    for x1 in range(1, m):
        for x2 in range(x1, m+1):
            for y1 in range(n):
                for y2 in range(y1, n+1):
                    total = (
                        prefix_sum[x2][y2] - prefix_sum[x2][y1-1] - 
                        prefix_sum[x1-1][y2] + prefix_sum[x1-1][y1-1]
                        )
                    if total == target:
                        count += 1

    return count

In [5]:
submatrix_sum(matrix=[[1, 2], [3, 4]], target=4)

1

In [6]:
def submatrix_sum(matrix: List[List[int]], target: int):
    m, n = len(matrix), len(matrix[0])

    prefix_sum = [[0] * (n+1) for _ in range(m+1)]
    for i in range(1, m+1):
        for j in range(1, n+1):
            prefix_sum[i][j] = (
                prefix_sum[i - 1][j] + 
                prefix_sum[i][j - 1] - 
                prefix_sum[i-1][j-1] +
                matrix[i-1][j-1]
            )
    
    # Now that we have computed all prefix sums, 
    # we go through each submatrix given by top left corner (x1, y1)
    # and bottom right corner (x2, y2) and check if the sum of elements
    # in this matrix sum to the target; using the prefix sum computed.
    count = 0
    
    for x1 in range(0, m):
        for x2 in range(x1, m+1):
            # We will use a dictionary to map the prefix sum to the count
            hashmap = defaultdict(int)  
            hashmap[0] = 1
            for y in range(1, n+1):                
                total = prefix_sum[x2][y] - prefix_sum[x1-1][y]
                count += hashmap[total]
                hashmap[total + target] += 1

    return count

In [7]:
submatrix_sum(matrix=[[1, 2], [3, 4]], target=4)

2

## Kadane's algorithm

Given an input array find the subarray with the maximum sum and 
1. Return the maximum sum
2. Return the subarray with the maximum sum

At compute an array *M* whose element at index *i* tells the maximum subarray that ends at the index *i*. 
Now this can depend on the previous index *i-1* as follows: either we extend the previous subarray by 
including this element at index *i*, or we start a new subarray with the solo element. The value of 
*M(i)* will be whichever is the max. The maximum subarray sum will be the max element of *M*. 

In [8]:
def max_subarray_sum_varient_1(nums: List[int]) -> int:
    subarray_end_sum = nums[0]
    global_max = nums[0]

    for i in range(1, len(nums)):
        subarray_end_sum = max(nums[i], subarray_end_sum + nums[i])
        global_max = max(global_max, subarray_end_sum)
    return global_max


def max_subarray_sum_varient_2(nums: List[int]) -> Tuple[int, int]:
    subarray_end_sum = [nums[0]]
    global_max = nums[0]
    global_max_index = 0

    for j in range(1, len(nums)):
        curr_end_sum = max(nums[j], subarray_end_sum[-1] + nums[j])
        subarray_end_sum.append(curr_end_sum)

        if curr_end_sum > global_max:
            global_max_index = j
            global_max = curr_end_sum

    # Since we already have the index of the end of the max
    # subarray stored in global_max_index, all we need to do to 
    # find the start index of the subarray is to traverse in the 
    # backward direction and check if it was included or not.
    i = global_max_index
    while subarray_end_sum[i] == global_max:
        global_max -= nums[i]
        i -= 1
        
    return i+1, global_max_index

In [9]:
max_subarray_sum_varient_2([-2])

(0, 0)

## Largest rectangle

Given an array of heights that can be thought as histogram, we want to find the rectangle with the largest area.

<img src="../images/Screenshot 2024-02-25 at 1.11.03 PM.png" width="300" height="auto">

In [10]:
def max_rectangle(heights: List[int]) -> int:
    my_stack = []
    max_area = 0

    for i, height in enumerate(heights):
        last_index = i
        while my_stack and my_stack[-1][0] > height:
            pop_height, pop_index = my_stack.pop()
            max_area = max(max_area, pop_height * (i - pop_index))
            last_index = pop_index
        my_stack.append((height, last_index))

    for pop_height, pop_index in my_stack:
        max_area = max(max_area, pop_height * (len(heights) - pop_index))

    return max_area
    


In [11]:
max_rectangle([1, 2, 3, 4, 5])

9

In [12]:
def maximal_rectangle(matrix):
    n_rows, n_cols = len(matrix), len(matrix[0])

    last_row = [0] * n_cols

    max_area = 0
    for row in range(n_rows):
        # For each row, we compute the heights with this row as the base.
        this_row = []
        for column in range(n_cols):
            if int(matrix[row][column]) == 0:
                this_row.append(0)
            else:
                this_row.append(last_row[column] + int(matrix[row][column]))

        # We compute the maximal area using the max rectangle in a histogram solution.
        max_area = max(max_area, max_rectangle(this_row))
        last_row = this_row

    return max_area


In [13]:
maximal_rectangle(matrix = [["1","0","1","0","0"],["1","0","1","1","1"],["1","1","1","1","1"],["1","0","0","1","0"]])

6

In [20]:
def min_window(s: str, t: str):
    if len(t) == 0:
        return ""
    
    # We create a hashmap indicating what characters we need and how many.
    hashmap_t = defaultdict(int)
    for char in t:
        hashmap_t[char] += 1

    have, need = 0, len(hashmap_t)
    index_range, index_length = [-1, -1], float('inf')
    left_index = 0
    hashmap_s = defaultdict(int)

    for i, char_s in enumerate(s):
        if char_s in hashmap_t:
            hashmap_s[char_s] += 1
        
            if hashmap_s[char_s] == hashmap_t[char_s]:
                have += 1

            # We start moving the left pointer, in hope of finding 
            # a range with smaller length.
            while have == need:
                # Update the result
                curr_length = i - left_index + 1
                if curr_length < index_length:
                    index_range = [left_index, i]
                    index_length = curr_length

                # If we stumble upon a required character, we update 
                # the count.
                last_char = s[left_index]
                if last_char in hashmap_t:
                    hashmap_s[last_char] -= 1

                    if hashmap_s[last_char] < hashmap_t[last_char]:
                        have -= 1

                left_index += 1

    return s[index_range[0]:index_range[1]+1] if index_length != float('inf') else ""

In [21]:
min_window(s = "ADOBECODEBANC", t = "ABBC")

'BECODEBA'

In [25]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [67]:
def dfs_greater(root, key):
    # We want elements bigger then the key.
    if (root is not None):
        if root.val < key:
            return dfs_greater(root.right, key)
        if root.val >= key:
            # Then we know that the right subtree will be bigger then the key.
            left = dfs_greater(root.left, key)
            right = dfs_greater(root.right, key)
            return left + right + root.val
    return 0

def dfs_smaller(root, key):
    # We want elements smaller then the key.
    if (root is not None):
        if root.val <= key:
            left = dfs_smaller(root.left, key)
            right = dfs_smaller(root.right, key)
            return left + right + root.val
        if root.val >= key:
            # Then we know that the right subtree will be bigger then the key.
            return dfs_smaller(root.left, key)
    return 0
        

def sum_nodes(root, left, right):
    if root.val >= right:
        return sum_nodes(root.left, left, right)
    elif root.val <= left:
        return sum_nodes(root.right, left, right)
    else:
        return dfs_greater(root.left, left) + dfs_smaller(root.right, right) + root.val

In [68]:
# BST formation
root = Node(4)
root.left = Node(2)
root.right = Node(6)
root.left.left = Node(1)
root.left.right = Node(3)
root.right.left = Node(5)
root.right.right = Node(7)

In [69]:
sum_nodes(root, 2, 5)

14

In [76]:
def dfs_stack(node):
    stack = []
    while True:
        while node is not None:
            stack.append(node)
            node = node.left

        if len(stack) > 0:
            pop_node = stack.pop()
            print(pop_node.val)
            node = pop_node.right
        else:
            break

In [77]:
dfs_stack(root)

1
2
3
4
5
6
7


Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000

In [105]:
def longest_palindrome_substring(s: str) -> str:
    m = len(s)

    dp_lengths = [[0] * m for _ in range(m)]

    # Base case: every character is a palindrome is length 1
    for i in range(m):
        dp_lengths[i][i] = 1

    # We want dp_lengths[0][m], where dp_length[i][j] is the length of the longest
    # panlindrom that is contained within the indices i and j.
    for i in range(m-2, -1, -1):
        for j in range(i+1, m):
            if (s[i] == s[j]) and (dp_lengths[i+1][j-1] == (j - i - 1)):
                updated_length = 2 + dp_lengths[i+1][j-1]
            else:
                updated_length = dp_lengths[i+1][j-1]
            dp_lengths[i][j] = max(dp_lengths[i+1][j], dp_lengths[i][j-1], updated_length)

    # To get the substring we traverse backwards, either increasing i 
    # or decreasing j till the max length remains the same.
    max_length = dp_lengths[0][m-1]

    start, end = 0, m-1
    while start < end:
        if dp_lengths[start+1][end] == max_length:
            start += 1
        elif dp_lengths[start][end-1] == max_length:
            end -= 1
        else:
            break

    return s[start: end+1]
        


In [106]:
longest_palindrome_substring("a")

'a'